In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv(dotenv_path="../../keys/.env")

True

In [2]:
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")

In [3]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# List of PDF file paths
pdf_dir = "../../data/pdfs"
pdf_files = [os.path.join(pdf_dir, file) for file in os.listdir(pdf_dir) if file.endswith(".pdf")]

# URLs to load
urls = [
    "https://www.math-datascience.nat.fau.de/im-studium/masterstudiengaenge/master-data-science/",
    "https://www.fau.eu/studiengang/data-science-bsc/",
    "https://www.fau.eu/studiengang/data-science-msc/",
]

# Initialize an empty list to hold all document splits
all_doc_splits = []

# Define the text splitter with a chunk size of 1000 characters and overlap of 200
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, 
    chunk_overlap=200)

### Process PDF files ###
for pdf_file in pdf_files:
    # Load the PDF
    pdf_loader = PyPDFLoader(pdf_file)
    documents = pdf_loader.load()
    
    # Split the documents into chunks
    doc_splits = text_splitter.split_documents(documents)
    
    # Add the splits to the overall list
    all_doc_splits.extend(doc_splits)

### Process Web URLs ###
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Split the web documents into chunks
web_doc_splits = text_splitter.split_documents(docs_list)

# Add the web document splits to the overall list
all_doc_splits.extend(web_doc_splits)

# Now, all_doc_splits contains the splits from both the PDF files and the web documents.

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
print(len(all_doc_splits))

175


In [6]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_community.vectorstores import FAISS


vectorstore = FAISS.from_documents(documents=all_doc_splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [7]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate three 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative three questions separated by one empty line, give only questions and nothing else. Original question: {question}"""

prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives 
    | ChatGroq(model_name="llama-3.3-70b-versatile") 
    | StrOutputParser() 
    | (lambda x: x.split("\n\n"))
)

In [9]:
questions = [
    "Will the language of instruction of my Bachelor’s degree suffice to demonstrate my proficiency in English for admission in MSc Data Science at FAU Erlangen?",
    "Is it possible to defer my admission to a future semester?",
    "What are the consequences of failing an examination three times?",
    "Who is eligible to supervise my Master’s thesis?",
    "How many ECTS credits are required before I can start writing my Master’s thesis?",
    "What are the admission requirements of MSc Data Science?",
    "What is the teaching language of MSc Data Science?",
    "What is the teaching language of BSc Data Science?",
    "What are the next steps if I get an admission?",
    "Is German required for MSc Data Science?",
    "As your application period will start on 15 February 2024 for the winter semester 2024/25, I have received an offer letter for the summer semester 2024. Do I have to reapply to defer my admission? Can I reapply with my existing account on the Campo portal, which I used for the summer semester, or do I have to open a new account to reapply for the winter 2024/25?",
    "Hello, I have applied to a Master's degree program in Data Science (both part-time and full-time) at FAU. Unfortunately, I have not added my previous offer letter (offer letter is attached below). However, the application portal is not letting me add this offer letter now as it says I cannot change it because my other applications are in process. How do I add this now? I have tried to contact the Masterbüro and admission office, but I have not received any response yet!",
    "I just wanted to know if the course 'Trustworthy Artificial Intelligence' can be taken as a Master Seminar in my course. On Campo, I find it mentioned as both a Master Seminar and an application subject (screenshot attached). Just to clarify before taking the subject.",
    "I am currently pursuing a Master's degree in Data Science. I have registered for the course 'Machine Learning for Engineers 1' for the upcoming examination. Is it eligible to be counted towards the requirements of my degree program? If it is, could you please inform me under which category it falls? I would greatly appreciate your guidance on this matter.",
    "What exactly defines the completion of a degree? Is it the transcript showing 120 ECTS? Let’s say I complete my thesis in September, but the transcript gets updated exactly on 15th October (for example). According to the university, will I be de-enrolled at the end of the Winter semester 2025? But if I apply for de-enrollment myself, can I be de-enrolled at any time during the winter semester? Hope I understood the information correctly.",
    "Sir/Madam, I am a prospective student of your esteemed university from India. I would like to join the Master's program in Data Science (English-taught) in 2024. I have some doubts regarding the full-time study program: 1) What is the duration of a single semester? 2) How many hours of class should I take in a week? 3) If I opt for the full-time study program, would I be able to do part-time jobs? Eagerly waiting for your reply.",
    "I'm a final-year Electrical Engineering student at NUST, Pakistan, interested in the Master's in Data Science program. Having taken relevant coursework in statistics and computer science, along with online courses, I'm curious about my eligibility. My transcript is attached for review.",
    "I have failed two courses: 'Mathematical Foundations of Artificial Intelligence' and 'Neural Networks and Data Analytics - Artificial Intelligence I'. Both of them have not been registered for the exam automatically. Last semester, my failed courses were registered automatically, but this time, it doesn't look like that. Please clarify!",
    "I am a Master's in Data Science student keen to enroll in a Computer Vision Seminar. This seminar is closely related to my academic and career goals. I also got enrolled in this course. However, I couldn't find it in the current course catalog for our program. I kindly request the addition of the Computer Vision Seminar to our course offerings. I believe it will greatly benefit students interested in this field. Your consideration is greatly appreciated.",
    "I am from Data Science, and I am writing to seek your guidance regarding the completion of my major module requirements. I have completed all my credits except for 2.5 ECTS in the major module. In light of this, I would like to ask whether I can fulfill this requirement by taking only a 2.5 ECTS subject, or if I can also consider enrolling in a 5 ECTS subject to meet this requirement. I would appreciate your advice on the best course of action to ensure I complete my credits in a timely and effective manner."
]

In [10]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

In [11]:
answers = []
for question in questions:
    retrieval_chain = generate_queries | retriever.map() | reciprocal_rank_fusion
    docs = retrieval_chain.invoke({"question":question})

    from operator import itemgetter
    from langchain_openai import ChatOpenAI
    from langchain_core.runnables import RunnablePassthrough

    # RAG
    template = """Answer the following question based on this context:

    {context}

    Question: {question}
    """

    prompt = ChatPromptTemplate.from_template(template)

    final_rag_chain = (
        {"context": retrieval_chain, 
        "question": itemgetter("question")} 
        | prompt
        | ChatGroq(model_name="llama-3.3-70b-versatile")
        | StrOutputParser()
    )

    ans = final_rag_chain.invoke({"question":question})
    answers.append(ans)

C:\Users\ekaan\AppData\Local\Temp\ipykernel_11812\239430494.py:26: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  (loads(doc), score)


In [12]:
answers

["Yes, if your Bachelor's degree was taught in English, you can apply with a certificate from your university stating that the language of instruction for your degree has been in English. This will be sufficient to prove your English language proficiency for admission to the MSc Data Science program at FAU Erlangen.",
 'Yes, it is possible to defer your admission to a future semester. According to the provided context, specifically in the table of contents of the \'msc-datascience_faq.pdf\' document, there is a question "Is it possible to defer my admission to an upcoming semester?" which is listed on page 1. However, the actual answer to this question is not provided in the given context. \n\nTo get the correct answer, you would need to refer to the actual content of the FAQ document, specifically the section that addresses deferring admission. Unfortunately, based on the information provided, we cannot give a definitive answer to whether deferring admission is allowed and under what 

In [13]:
Ground_truth = [
    "Yes, if your Bachelor’s degree was taught in English then you can apply with a certificate from your university stating that the language of instruction for your degree has been in English.",
    "Yes, it is possible to defer your admission. Just re-apply for the next semester and upload the current admission letter you received. You will be granted admission provided that there are no significant changes to the regulations and selection criteria.",
    "After three unsuccessful attempts for an examination, the module will be counted as not being passed. This state cannot be changed, and you cannot pass this module in the future. If the module is an elective mandatory module (core elective module in Campo), you can choose another module and continue your studies. If the module is a core module of your study course (Mathematics of Learning, Selected Topics of Mathematics of Learning, or Deep Learning), you cannot continue your studies in this program at FAU.",
    "To start your Master’s thesis, you need an official supervisor from FAU. The person must be a professor or at least have completed a habilitation ('Dr. habil.' or 'PD') and must be a member of one of the following departments: Data Science, Mathematics, Computer Science, or Artificial Intelligence in Biomedical Engineering (AIBE). Professors who are secondary members in these departments can also supervise. Your official supervisor is also the first examiner of your thesis and should propose a second examiner from any department at FAU. Additional guidance from PhD students is possible, but they cannot be official supervisors.",
    "It is highly recommended that you complete 75 ECTS of your study curriculum before choosing a Master’s seminar topic. You should complete all core modules and major modules first. Your thesis topic should ideally align with your specialization and Master’s seminar, following §54 of the examination regulations ('Mentoring') and your individual study agreement.",
    "A completed B.Sc. degree in Mathematics, Industrial Mathematics, Mathematical Economy, Computer Science, Data Science, or Physics from FAU or an equivalent domestic or international degree that aligns with the competence profile of the program. A GPA of 2.5 or better (German grading system) is required. Applicants with a GPA between 2.6 and 2.8 will be invited for an online interview assessing their knowledge of calculus, linear algebra, algorithms, and data structures. English proficiency at B2 CEFR or proof of English-taught education is required.",
    "Completely in English.",
    "Completely in German.",
    "After receiving your admission letter, you must enroll for the next semester by sending certified documents via postal service to the Student Records Office. The enrollment fee should be transferred in advance, especially if transferring from abroad (recommended two weeks before). After enrollment, you gain access to online teaching resources and platforms like StudOn and Campo.",
    "No, but solid German knowledge (B1) is recommended for daily life, internships, and working student jobs. However, a certificate is not required for the application.",
    "You can reapply using your existing Campo account.",
    "Due to the abolition of the 60 ECTS regulation from the winter semester 2022/23, the examination regulations of the Data Science program changed. The letter of admission from winter semester 2021/22 will no longer be recognized. Your application will be completely re-examined and re-evaluated, so uploading the previous offer letter makes no difference.",
    "As you found on Campo, the seminar can be counted as a Master Seminar.",
    "This course is not recognized for the MSc Data Science study program. Therefore, even if you attend this course, you will not be able to accredit any ECTS.",
    "To complete your degree, you need 120 ECTS, including all core, major, minor, application modules, seminar, and thesis. If you complete 120 ECTS by September 2024 and your supervisor submits your thesis grade before the end of September, you will be deregistered at the end of the summer semester 2024. If you complete it in October 2024, you will be deregistered at the end of the winter semester 2024/25. You can also choose to de-register earlier if desired.",
    "1) The duration of one semester is 6 months. 2) There is no restriction on study hours per week. You can take subjects at your comfort, but 30 ECTS per semester is recommended (1 ECTS = 1 hour of study). 3) Yes, you can work part-time. A student can work up to 20 hours per week with a part-time student job.",
    "The admissions committee evaluates your profile only after you apply via the Campo portal.",
    "The automatic re-registration for second attempts in examinations is done manually by the examination office.",
    "I recommend sending a course description for the seminar, including details like the module number and exact name in Campo, to the examination committee and asking if the seminar can be included in one of the module groups of the MSc Data Science program.",
    "You can complete a 5 ECTS module instead of a 2.5 ECTS module. Just ensure that you meet at least the 2.5 ECTS requirement."
]

In [14]:
from datasets import Dataset 
from ragas.metrics import faithfulness, answer_correctness
from ragas import evaluate

In [15]:

data_samples = {
    
    'question': questions,
    
    "answer": answers,
    
    'ground_truth': Ground_truth
}

dataset = Dataset.from_dict(data_samples)
score = evaluate(dataset,metrics=[answer_correctness])
score.to_pandas()

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

,user_input,response,reference,answer_correctness
0,Will the language of instruction of my Bachelo...,"Yes, if your Bachelor's degree was taught in E...","Yes, if your Bachelor’s degree was taught in E...",0.874733
1,Is it possible to defer my admission to a futu...,"Yes, it is possible to defer your admission to...","Yes, it is possible to defer your admission. J...",0.323462
2,What are the consequences of failing an examin...,"According to the text, if a student fails an e...",After three unsuccessful attempts for an exami...,0.993366
3,Who is eligible to supervise my Master’s thesis?,"According to the provided context, your Master...","To start your Master’s thesis, you need an off...",0.456881
4,How many ECTS credits are required before I ca...,It is highly recommended that you complete 75 ...,It is highly recommended that you complete 75 ...,0.737872
5,What are the admission requirements of MSc Dat...,The admission requirements for the MSc Data Sc...,"A completed B.Sc. degree in Mathematics, Indus...",0.660316
6,What is the teaching language of MSc Data Scie...,The teaching language of the MSc Data Science ...,Completely in English.,0.965666
7,What is the teaching language of BSc Data Scie...,The teaching language of the BSc Data Science ...,Completely in German.,0.967110
8,What are the next steps if I get an admission?,"Unfortunately, the provided context does not e...","After receiving your admission letter, you mus...",0.211262
9,Is German required for MSc Data Science?,"According to the provided context, German is n...","No, but solid German knowledge (B1) is recomme...",0.762430


In [16]:
print(sum(score.to_pandas()["answer_correctness"]) / len(score.to_pandas()["answer_correctness"]))

0.6842794052280639
